In [4]:
import os
from PIL import Image

In [5]:
dataset_root = "./dataset"  # e.g. "./brain_tumor_dataset"
output_images_dir = "yolo_images"
output_labels_dir = "yolo_labels"

In [6]:
label_map = {
    "glioma": 0,
    "meningioma": 1,
    "notumor": 2,
    "pituitary": 3,
}

In [7]:
for split in ["Training", "Testing"]:
    for subdir in label_map:
        os.makedirs(os.path.join(output_images_dir, split, subdir), exist_ok=True)
        os.makedirs(os.path.join(output_labels_dir, split, subdir), exist_ok=True)


In [9]:
from PIL import Image
import os

for split in ["Training", "Testing"]:
    for class_name in label_map:
        class_dir = os.path.join(dataset_root, split, class_name)
        if not os.path.exists(class_dir):
            continue
        class_id = label_map[class_name]

        for filename in os.listdir(class_dir):
            if not filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                continue

            # Open image to get dimensions
            image_path = os.path.join(class_dir, filename)
            image = Image.open(image_path)

            # Convert to RGB if needed
            if image.mode in ("RGBA", "P"):
                image = image.convert("RGB")

            width, height = image.size

            # Create YOLO annotation (full image box)
            yolo_line = f"{class_id} 0.5 0.5 1.0 1.0\n"

            # Save image to output directory
            new_image_path = os.path.join(output_images_dir, split, class_name, filename)
            os.makedirs(os.path.dirname(new_image_path), exist_ok=True)
            image.save(new_image_path)

            # Save label file
            label_filename = os.path.splitext(filename)[0] + ".txt"
            label_path = os.path.join(output_labels_dir, split, class_name, label_filename)
            os.makedirs(os.path.dirname(label_path), exist_ok=True)
            with open(label_path, "w") as f:
                f.write(yolo_line)
